In [ ]:
import numpy as np
import pandas as pd
import requests, json


In [ ]:
csvframe = pd.read_csv('loanphys_.csv', delimiter=";", decimal=",")
csvframe['REPORT_DATE'] = pd.to_datetime(csvframe['REPORT_DATE'])
csvframe = csvframe.sort_values (by=['DEAL_ID', 'REPORT_DATE'])
csvframe['DEBT_NM_AMT'] = csvframe['DEBT_NM_AMT'].astype(float)
csvframe['PROVISION_RATE'] = csvframe['PROVISION_RATE'].astype(float)
csvframe['PROVISION_DEBT_AMT'] = csvframe['PROVISION_DEBT_AMT'].astype(float)
csvframe = csvframe.fillna(0)
csvframe.index = list(range(0, len(csvframe), 1))
csvframe.head(12)


,REPORT_DATE,DEAL_ID,DEAL_PARENT_ID,DEAL_TYPE,BALACC_DEBT_CODE,BALACC_OVERDUE_CODE,PRODUCT_NAME,CURRENCY_CODE,BEGIN_DATE,END_FIRST_DATE,END_PLAN_DATE,TERM_OVERDUEDEBT_DAY_CNT,TERM_OVERDUEINTEREST_DAY_CNT,DEAL_AMT,DEBT_NM_AMT,DEBT_RUR_AMT,OVERDUE_NM_AMT,OVERDUE_RUR_AMT,INTEREST_RATE,INTEREST_NM_AMT,INTEREST_RUR_AMT,OVERDUEINTEREST_NM_AMT,OVERDUEINTEREST_RUR_AMT,RISKGROUP_CODE,PROVISION_RATE,PROVISION_DEBT_AMT,PROVISION_OVERDUE_AMT,PROVISION_INTEREST_AMT,PROVISION_OVERDUEINTEREST_AMT
0,2020-03-31,6943416,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,"2,924.00000","2,536.00000","41,000.00000",0.00000,0.00000,"21,463.40000","21,463.40000",0.00000,0.00000,0.00000,"4,523.44000","4,523.44000",5,100.00000,0.00000,"21,463.40000",0.00000,"4,523.44000"
1,2020-04-30,6943416,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,"2,954.00000","2,566.00000","41,000.00000",0.00000,0.00000,"21,463.40000","21,463.40000",0.00000,0.00000,0.00000,"4,523.44000","4,523.44000",5,100.00000,0.00000,"21,463.40000",0.00000,"4,523.44000"
2,2020-05-31,6943416,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,"2,985.00000","2,597.00000","41,000.00000",0.00000,0.00000,"21,463.40000","21,463.40000",0.00000,0.00000,0.00000,"4,523.44000","4,523.44000",5,100.00000,0.00000,"21,463.40000",0.00000,"4,523.44000"
3,2020-06-30,6943416,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,"3,015.00000","2,627.00000","41,000.00000",0.00000,0.00000,"21,463.40000","21,463.40000",0.00000,0.00000,0.00000,"4,523.44000","4,523.44000",5,100.00000,0.00000,"21,463.40000",0.00000,"4,523.44000"
4,2020-07-31,6943416,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,"3,046.00000","2,658.00000","41,000.00000",0.00000,0.00000,"21,463.40000","21,463.40000",0.00000,0.00000,0.00000,"4,523.44000","4,523.44000",5,100.00000,0.00000,"21,463.40000",0.00000,"4,523.44000"
5,2020-08-31,6943416,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,24.05.2011,24.05.2013,24.05.2013,"3,077.00000","2,689.00000","41,000.00000",0.00000,0.00000,"21,463.40000","21,463.40000",0.00000,0.00000,0.00000,"4,523.44000","4,523.44000",5,100.00000,0.00000,"21,463.40000",0.00000,"4,523.44000"
6,2020-03-31,6944919,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,29.12.2010,28.12.2012,28.12.2012,"2,631.00000","2,569.00000","88,000.00000",0.00000,0.00000,"57,469.55000","57,469.55000",0.00000,0.00000,0.00000,"4,273.67000","4,273.67000",5,100.00000,0.00000,"57,469.55000",0.00000,"4,273.67000"
7,2020-04-30,6944919,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,29.12.2010,28.12.2012,28.12.2012,"2,661.00000","2,599.00000","88,000.00000",0.00000,0.00000,"57,469.55000","57,469.55000",0.00000,0.00000,0.00000,"4,273.67000","4,273.67000",5,100.00000,0.00000,"57,469.55000",0.00000,"4,273.67000"
8,2020-05-31,6944919,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,29.12.2010,28.12.2012,28.12.2012,"2,692.00000","2,630.00000","88,000.00000",0.00000,0.00000,"57,469.55000","57,469.55000",0.00000,0.00000,0.00000,"4,273.67000","4,273.67000",5,100.00000,0.00000,"57,469.55000",0.00000,"4,273.67000"
9,2020-06-30,6944919,0.00000,0.00000,0.00000,"45,815.00000",Пенсионеры,810,29.12.2010,28.12.2012,28.12.2012,"2,722.00000","2,660.00000","88,000.00000",0.00000,0.00000,"57,469.55000","57,469.55000",0.00000,0.00000,0.00000,"4,273.67000","4,273.67000",5,100.00000,0.00000,"57,469.55000",0.00000,"4,273.67000"


In [ ]:
#досоздать строку в csvframe с датой 31.07.2020 для id = 62909499
copy = csvframe.iloc[1432].copy()
copy['REPORT_DATE'] = pd.to_datetime("2020-07-31")
csvframe.loc[len(csvframe)] = copy

2020-08-31 00:00:00
1432


In [ ]:
csvframe.columns

Index(['REPORT_DATE', 'DEAL_ID', 'DEAL_PARENT_ID', 'DEAL_TYPE', 'BALACC_DEBT_CODE', 'BALACC_OVERDUE_CODE',
       'PRODUCT_NAME', 'CURRENCY_CODE', 'BEGIN_DATE', 'END_FIRST_DATE', 'END_PLAN_DATE', 'TERM_OVERDUEDEBT_DAY_CNT',
       'TERM_OVERDUEINTEREST_DAY_CNT', 'DEAL_AMT', 'DEBT_NM_AMT', 'DEBT_RUR_AMT', 'OVERDUE_NM_AMT', 'OVERDUE_RUR_AMT',
       'INTEREST_RATE', 'INTEREST_NM_AMT', 'INTEREST_RUR_AMT', 'OVERDUEINTEREST_NM_AMT', 'OVERDUEINTEREST_RUR_AMT',
       'RISKGROUP_CODE', 'PROVISION_RATE', 'PROVISION_DEBT_AMT', 'PROVISION_OVERDUE_AMT', 'PROVISION_INTEREST_AMT',
       'PROVISION_OVERDUEINTEREST_AMT'],
      dtype='object')

In [ ]:
exchange_rate = pd.DataFrame(sorted(list(set((csvframe['REPORT_DATE'])))), columns = ["date"]) #табличка с курсом валют по датам из csvframe
print(set(csvframe['CURRENCY_CODE'])) #какие есть валюты


{840, 978, 810}


In [ ]:
exchange_rate['810'] = [1, 1, 1, 1, 1, 1]
exchange_rate['840'] = [77.7325, 73.6894, 70.7520, 69.9513, 73.3633, 74.6382]
exchange_rate['978'] = [85.7389, 80.0488, 78.5489, 78.6812, 86.2532, 88.7448]
exchange_rate['840'] = exchange_rate['840'].astype(float)
exchange_rate['810'] = exchange_rate['810'].astype(float)
exchange_rate['978'] = exchange_rate['978'].astype(float)

In [ ]:
exchange_rate.set_index("date", inplace=True)
exchange_rate

,810,840,978
date,,,
2020-03-31,1.00000,77.73250,85.73890
2020-04-30,1.00000,73.68940,80.04880
2020-05-31,1.00000,70.75200,78.54890
2020-06-30,1.00000,69.95130,78.68120
2020-07-31,1.00000,73.36330,86.25320
2020-08-31,1.00000,74.63820,88.74480


Функция, которая возвращает курс $i$-й день:

In [ ]:
def currency(i): 
  c = 1
  if csvframe['CURRENCY_CODE'][i] == 840:
    c = exchange_rate.loc[csvframe['REPORT_DATE'][i]][1]
  elif csvframe['CURRENCY_CODE'][i] == 978:
    c = exchange_rate.loc[csvframe['REPORT_DATE'][i]][2]
  return c


Функции, которые возвращают значения остатка, просрочки, ставки, суммы начисленных и просроченных процентов, суммы резерва под возможные потери ПОД, суммы резерва по процентам ОД, суммы резерва по процентам ПОД, суммы резерва под возможные потери ОД соответственно на  $i$-й день: 

In [ ]:
def debt(i):
  return (csvframe['DEBT_NM_AMT'][i])

def overdue_debt(i):
  return csvframe['OVERDUE_NM_AMT'][i]

def provision_rate(i):
  return (csvframe['PROVISION_RATE'][i] / 100)

def interest(i):
  return csvframe['INTEREST_NM_AMT'][i]

def overdue_interest(i):
  return csvframe['OVERDUEINTEREST_NM_AMT'][i]

def provision_overdue(i):
  return csvframe['PROVISION_OVERDUE_AMT'][i]

def provision_interest(i):
  return csvframe['PROVISION_INTEREST_AMT'][i]

def provision_overdueinterest(i):
  return csvframe['PROVISION_OVERDUEINTEREST_AMT'][i]

def provision_debt_amt(i):
  return csvframe['PROVISION_DEBT_AMT'][i]

Если элемент elem больше 0, то plus = elem, minus = 0, иначе minus = elem, plus = 0

In [ ]:
def sign_append(plus, minus, elem):
  if (elem > 0):
    plus = elem
    minus = 0
  else:
    plus = 0
    minus = elem

Функция, которая возвращает сумму резерва под возможные потери ОД, под возможные потери ПОД, по процентам ОД, по процентам ПОД

In [ ]:
def sum_provision(i):
  return provision_interest(i) + provision_overdueinterest(i) + provision_debt_amt(i) + provision_overdue(i)


Создадим шаблон таблицы с показателями формирования и восстановления резерва.
Названия столбцов:

$\underline{Формирование \; резерва:}$

Столбец 'provision_form_dA' -- выдача новых ссуд (основной долг) \\
Столбец 'provision_form_dR' -- увеличение ставки резервирования (основной долг) \\
Столбец 'provision_form_dC' -- изменение курса валюты (основной долг) \\
Столбец 'provision_form_dO' -- прочие причины (основной долг) \\


$\underline{Восстановление \; резерва:}$

Столбец 'provision_rec_dA' -- погашение  ссуд (основной долг) \\
Столбец 'provision_rec_dR' -- уменьшение ставки резервирования (основной долг) \\
Столбец 'provision_rec_dC' -- изменение курса валюты (основной долг) \\
Столбец 'provision_rec_dO' -- прочие причины (основной долг) \\


In [ ]:
data = {pd.to_datetime('2020-04-30'): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        pd.to_datetime('2020-05-31'): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        pd.to_datetime('2020-06-30'): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        pd.to_datetime('2020-07-31'): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        pd.to_datetime('2020-08-31'): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] }

delta_provision = pd.DataFrame(data) #таблица для основного долга
delta_provision['parameter'] = ['sum_form', 'provision_form_dA', 'provision_form_dR', 'provision_form_dC', 'provision_form_dO', 
                      'sum_rec', 'provision_rec_dA', 'provision_rec_dR', 'provision_rec_dC', 'provision_rec_dO']

delta_provision.set_index("parameter", inplace=True)
delta_provision_all_id = {} #словарь с таблицами типа delta_provision для каждого DEAL_ID

delta_provision


,2020-04-30 00:00:00,2020-05-31 00:00:00,2020-06-30 00:00:00,2020-07-31 00:00:00,2020-08-31 00:00:00
parameter,,,,,
sum_form,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dA,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dR,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dC,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dO,0.00000,0.00000,0.00000,0.00000,0.00000
sum_rec,0.00000,0.00000,0.00000,0.00000,0.00000
provision_rec_dA,0.00000,0.00000,0.00000,0.00000,0.00000
provision_rec_dR,0.00000,0.00000,0.00000,0.00000,0.00000
provision_rec_dC,0.00000,0.00000,0.00000,0.00000,0.00000


Заполним таблицы и посчитаем суммарное dA, dR, dC, dOther для всего портфеля

In [ ]:
sum_delta_provision = delta_provision.copy() #таблица с изменениями резерва для всего портфеля
sum_delta_provision.loc['dA'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dR'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dC'] = [0.0, 0.0, 0.0, 0.0, 0.0]
sum_delta_provision.loc['dO'] = [0.0, 0.0, 0.0, 0.0, 0.0]

In [ ]:
len(csvframe)

6041

In [ ]:
i = 0
while (i != len(csvframe)):
    j = 0
    temp_table = delta_provision.copy() #создаем табличку с показателями
    deal_id = csvframe['DEAL_ID'][i]
    while (j+i+1 < len(csvframe) and 
           csvframe['DEAL_ID'][j+i] == csvframe['DEAL_ID'][j+1+i]):
      debt_nm_amt1 = debt(i+j) + overdue_debt(i+j)
      debt_nm_amt2 = debt(i+j+1) + overdue_debt(i+j+1)
      c1 = currency(i+j)   
      c2 = currency(i+j+1)
      provision_rate1 = provision_rate(i+j)
      provision_rate2 = provision_rate(i+j+1)
      date = csvframe['REPORT_DATE'][i+j+1]
      d_a = (debt_nm_amt2 - debt_nm_amt1) * provision_rate1 * c1
      d_c = (c2 - c1) * provision_rate1 * debt_nm_amt2
      d_r = (provision_rate2 - provision_rate1) * debt_nm_amt2 * c2
      d_other = sum_provision(i+j+1) - sum_provision(i+j) - d_a - d_c - d_r
      j += 1

      sum_delta_provision[date]['dA'] += d_a
      sum_delta_provision[date]['dR'] += d_r
      sum_delta_provision[date]['dC'] += d_c
      sum_delta_provision[date]['dO'] += d_other

      if d_a > 0:
        temp_table[date]['provision_form_dA'] = d_a
        temp_table[date]['sum_form'] += d_a
      else:
        temp_table[date]['provision_rec_dA'] = d_a
        temp_table[date]['sum_rec'] += d_a
      if d_r > 0:
        temp_table[date]['provision_form_dR'] = d_r
        temp_table[date]['sum_form'] += d_r
      else:
        temp_table[date]['provision_rec_dR'] = d_r
        temp_table[date]['sum_rec'] += d_r
      if d_c > 0:
        temp_table[date]['provision_form_dC'] = d_c
        temp_table[date]['sum_form'] += d_c
      else:
        temp_table[date]['provision_rec_dC'] = d_c
        temp_table[date]['sum_rec'] += d_c
      if d_other > 0:
        temp_table[date]['provision_form_dO'] = d_other
        temp_table[date]['sum_form'] += d_other
      else:
        temp_table[date]['provision_rec_dO'] = d_other
        temp_table[date]['sum_rec'] += d_other
    delta_provision_all_id[deal_id] = temp_table #добавляем табличку с показателями в словарь
    i += (j+1)


Получили dA, dR, dC, dOther для портфеля на каждую из дат. Теперь составим таблицу:

In [ ]:
def count_parameter(sum_delta_provision, date):
  d_a = sum_delta_provision[date]['dA']
  d_c = sum_delta_provision[date]['dC']
  d_r = sum_delta_provision[date]['dR']
  d_other = sum_delta_provision[date]['dO']
  if d_a > 0:
    sum_delta_provision[date]['provision_form_dA'] = d_a
    sum_delta_provision[date]['sum_form'] += d_a
  else:
    sum_delta_provision[date]['provision_rec_dA'] = d_a
    sum_delta_provision[date]['sum_rec'] += d_a
  if d_r > 0:
    sum_delta_provision[date]['provision_form_dR'] = d_r
    sum_delta_provision[date]['sum_form'] += d_r
  else:
    sum_delta_provision[date]['provision_rec_dR'] = d_r
    sum_delta_provision[date]['sum_rec'] += d_r
  if d_c > 0:
    sum_delta_provision[date]['provision_form_dC'] = d_c
    sum_delta_provision[date]['sum_form'] += d_c
  else:
    sum_delta_provision[date]['provision_rec_dC'] = d_c
    sum_delta_provision[date]['sum_rec'] += d_c
  if d_other > 0:
    sum_delta_provision[date]['provision_form_dO'] = d_other
    sum_delta_provision[date]['sum_form'] += d_other
  else:
    sum_delta_provision[date]['provision_rec_dO'] = d_other
    sum_delta_provision[date]['sum_rec'] += d_other

In [ ]:
count_parameter(sum_delta_provision, pd.to_datetime('2020-04-30'))
count_parameter(sum_delta_provision, pd.to_datetime('2020-05-31'))
count_parameter(sum_delta_provision, pd.to_datetime('2020-06-30'))
count_parameter(sum_delta_provision, pd.to_datetime('2020-07-31'))
count_parameter(sum_delta_provision, pd.to_datetime('2020-08-31'))

In [ ]:
sum_delta_provision.drop(["dA", "dR", "dC", "dO"], inplace = True)
pd.options.display.float_format ='{:.5f}'.format
sum_delta_provision

,2020-04-30 00:00:00,2020-05-31 00:00:00,2020-06-30 00:00:00,2020-07-31 00:00:00,2020-08-31 00:00:00
parameter,,,,,
sum_form,7356428.68604,0.00000,2392531.98547,17993834.31307,5864263.17476
provision_form_dA,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dR,7356428.68604,0.00000,2392531.98547,5001066.43469,0.00000
provision_form_dC,0.00000,0.00000,0.00000,6207633.66841,2228724.60539
provision_form_dO,0.00000,0.00000,0.00000,6785134.20996,3635538.56937
sum_rec,-34746042.02604,-8163845.77000,-8369619.50547,-9483671.10307,-4487764.63476
provision_rec_dA,-10693451.36686,-3195096.61113,-5449336.07007,-9483671.10307,-4196794.28943
provision_rec_dR,0.00000,-923727.61755,0.00000,0.00000,-290970.34533
provision_rec_dC,-6506948.83014,-4033822.02895,-956814.77793,0.00000,0.00000


In [ ]:
sum_delta_provision['Показатель'] = ['1. Формирование резерва, в том числе:',
                                    '1.1. выдача новых ссуд',
                                    '1.2. увеличение ставки резервирования',
                                    '1.3. изменение курса валюты',
                                    '1.4. прочие причины',
                                    '2. Восстановление резерва, в том числе:',
                                    '2.1. погашение ссуд',
                                    '2.2. уменьшение ставки резервирования',
                                    '2.3. изменение курса валюты',
                                    '2.4. прочие причины']

sum_delta_provision.set_index("Показатель", inplace=True)
sum_delta_provision


,2020-04-30 00:00:00,2020-05-31 00:00:00,2020-06-30 00:00:00,2020-07-31 00:00:00,2020-08-31 00:00:00
Показатель,,,,,
"1. Формирование резерва, в том числе:",7356428.68604,0.00000,2392531.98547,17993834.31307,5864263.17476
1.1. выдача новых ссуд,0.00000,0.00000,0.00000,0.00000,0.00000
1.2. увеличение ставки резервирования,7356428.68604,0.00000,2392531.98547,5001066.43469,0.00000
1.3. изменение курса валюты,0.00000,0.00000,0.00000,6207633.66841,2228724.60539
1.4. прочие причины,0.00000,0.00000,0.00000,6785134.20996,3635538.56937
"2. Восстановление резерва, в том числе:",-34746042.02604,-8163845.77000,-8369619.50547,-9483671.10307,-4487764.63476
2.1. погашение ссуд,-10693451.36686,-3195096.61113,-5449336.07007,-9483671.10307,-4196794.28943
2.2. уменьшение ставки резервирования,0.00000,-923727.61755,0.00000,0.00000,-290970.34533
2.3. изменение курса валюты,-6506948.83014,-4033822.02895,-956814.77793,0.00000,0.00000


In [ ]:
table = delta_provision_all_id[62827302]
table

,2020-04-30 00:00:00,2020-05-31 00:00:00,2020-06-30 00:00:00,2020-07-31 00:00:00,2020-08-31 00:00:00
parameter,,,,,
sum_form,0.00737,0.00000,0.00882,1080.20866,399.31756
provision_form_dA,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dR,0.00000,0.00000,0.00000,0.00000,0.00000
provision_form_dC,0.00000,0.00000,0.00000,1080.20866,399.31538
provision_form_dO,0.00737,0.00000,0.00882,0.00000,0.00218
sum_rec,-1582.03737,-1194.33000,-496.71882,-234.90866,-247.81756
provision_rec_dA,-261.29003,-244.51617,-240.53557,-234.89996,-247.81756
provision_rec_dR,0.00000,0.00000,0.00000,0.00000,0.00000
provision_rec_dC,-1320.74735,-949.80477,-256.18324,0.00000,0.00000


In [ ]:
for i in range(len(csvframe)):
  if csvframe['DEAL_ID'][i] == 62827302:
    print(currency(i))
    print(csvframe['PROVISION_DEBT_AMT'][i] + csvframe['PROVISION_OVERDUE_AMT'][i] + csvframe['PROVISION_INTEREST_AMT'][i] + csvframe['PROVISION_OVERDUEINTEREST_AMT'][i])
    print(csvframe['DEBT_NM_AMT'][i] + csvframe['OVERDUE_NM_AMT'][i])
    print(csvframe['PROVISION_RATE'][i])
    print(csvframe['REPORT_DATE'][i])
    print()


77.7325
25653.93
66005.68
0.5
2020-03-31 00:00:00

73.6894
24071.9
65333.4
0.5
2020-04-30 00:00:00

70.752
22877.57
64669.76
0.5
2020-05-31 00:00:00

69.9513
22380.86
63989.82
0.5
2020-06-30 00:00:00

73.3633
23226.16
63318.21
0.5
2020-07-31 00:00:00

74.6382
23377.66
62642.62
0.5
2020-08-31 00:00:00

